In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch

In [15]:
class CNN1DModel(nn.Module):
    def __init__(self,
                 num_numerical_features: int,
                 hidden_size: int = 1024,
                 n_target: int = 1,
                 channel_1: int = 64,
                 channel_2: int = 128,
                 kernel_size: int = 5,
                 dropout_rate: float = 0.2):
        super().__init__()

        self.hidden_size = hidden_size  # Store hidden_size as an instance variable

        # 1. Expand 단계: Dense 레이어
        self.expand = nn.Sequential(
            nn.LayerNorm(num_numerical_features),
            nn.Dropout(dropout_rate),
            nn.Linear(num_numerical_features, hidden_size),
            nn.ReLU()
        )

        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=hidden_size // 16, 
                out_channels=channel_1, 
                kernel_size=kernel_size, 
                stride=1, 
                padding=kernel_size // 2
            ),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        # 3. Conv 블록 2
        self.conv2 = nn.Sequential(
            nn.Conv1d(
                in_channels=channel_1, 
                out_channels=channel_2, 
                kernel_size=kernel_size, 
                stride=1, 
                padding=kernel_size // 2
            ),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.AdaptiveAvgPool1d(output_size=16)
        )

        # 4. Flatten and Dense
        self.flatten = nn.Flatten()
        self.dense = nn.Sequential(
            nn.Linear(channel_2 * 16, 640),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(640, n_target)
        )

        # 추가된 Tanh
        self.tanh = nn.Tanh()

    def forward(self, x):
        # 1. Expand 단계
        x = self.expand(x)

        # 2. Reshape to match Conv input
        batch_size = x.size(0)
        seq_length = x.size(1) // (self.hidden_size // 16)  # Dynamically compute seq_length
        x = x.view(batch_size, self.hidden_size // 16, seq_length)

        # 3. Conv 블록 1
        x = self.conv1(x)

        # 4. Conv 블록 2
        x = self.conv2(x)

        # 4. Flatten and Dense
        x = self.flatten(x)
        x = self.dense(x)
        x = 5 * self.tanh(x)

        return x

In [2]:
def weighted_mse_loss(y_true, y_pred, weights):
    """
    Multi-target weighted MSE loss

    Args:
        y_true: target values (batch_size, n_targets)
        y_pred: predicted values (batch_size, n_targets)
        weights: weights for each target (batch_size, n_targets)
    """
    return torch.mean(weights * (y_true - y_pred)**2)

def weighted_r2_score(y_true, y_pred, weights):
    """
    Multi-target weighted R2 score

    Args:
        y_true: target values (batch_size, n_targets)
        y_pred: predicted values (batch_size, n_targets)
        weights: weights for each target (batch_size, n_targets)

    Returns:
        weighted R2 score (scalar)
    """
    # Ensure inputs are on CPU and converted to numpy
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    weights = weights.detach().cpu().numpy()

    weights = np.repeat(weights, y_true.shape[1], axis=1)

    # print(y_true.shape, y_pred.shape, weights.shape)
    # Calculate weighted means for each target
    weighted_mean = np.average(y_true, weights=weights, axis=0)

    # Calculate total sum of squares
    total_ss = np.sum(weights * (y_true - weighted_mean) ** 2, axis=0)

    # Calculate residual sum of squares
    residual_ss = np.sum(weights * (y_true - y_pred) ** 2, axis=0)

    # Calculate R2 score for each target
    r2_scores = 1 - (residual_ss / total_ss)

    # Return mean R2 score across all targets
    return np.mean(r2_scores)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, numerical_columns, target_columns, weight_columns=None):
        """
        Args:
            data: pandas DataFrame containing all features
            numerical_columns: list of column names for numerical features
            target_columns: list of target column names
            weight_columns: list of weight column names (optional)
        """
        self.numerical_features = torch.FloatTensor(data[numerical_columns].values)
        self.symbol = torch.LongTensor(data['symbol_id'].values)
        self.feature_09 = torch.LongTensor(data['feature_09'].values)
        self.feature_10 = torch.LongTensor(data['feature_10'].values)
        self.feature_11 = torch.LongTensor(data['feature_11'].values)
        self.time = torch.LongTensor(data['time_id'].values)

        # Multi-target 처리
        self.targets = torch.FloatTensor(data[target_columns].values)

        # 가중치 처리 (옵션)
        if weight_columns:
            self.weights = torch.FloatTensor(data[weight_columns].values)
        else:
            self.weights = torch.ones_like(self.targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'numerical_features': self.numerical_features[idx],
            'symbol_id': self.symbol[idx],
            'feature_09': self.feature_09[idx],
            'feature_10': self.feature_10[idx],
            'feature_11': self.feature_11[idx],
            'time_id': self.time[idx],
            'targets': self.targets[idx],
            'weights': self.weights[idx]
        }

def create_data_loaders(train_data, valid_data, numerical_columns,
                        target_columns, weight_columns=None,
                        batch_size=256, num_workers=4):
    """
    데이터로더를 생성하는 함수

    Args:
        train_data: 학습 데이터가 담긴 DataFrame
        valid_data: 검증 데이터가 담긴 DataFrame
        numerical_columns: 수치형 특성들의 컬럼명 리스트
        target_columns: 타겟 변수들의 컬럼명 리스트
        weight_columns: 가중치 컬럼명 리스트 (옵션)
        batch_size: 배치 크기
        num_workers: 데이터 로딩에 사용할 워커 수
    """

    # Dataset 객체 생성
    train_dataset = CustomDataset(train_data, numerical_columns, target_columns, weight_columns)
    valid_dataset = CustomDataset(valid_data, numerical_columns, target_columns, weight_columns)

    # DataLoader 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, valid_loader


def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    total_r2 = 0
    num_batches = len(train_loader)

    for batch_idx, batch in enumerate(train_loader):
        # 데이터를 디바이스로 이동
        numerical_features = batch['numerical_features'].to(device)
        symbol = batch['symbol_id'].to(device)
        feature_09 = batch['feature_09'].to(device)
        feature_10 = batch['feature_10'].to(device)
        feature_11 = batch['feature_11'].to(device)
        targets = batch['targets'].to(device)
        weights = batch['weights'].to(device)

        # Forward pass
        outputs = model(numerical_features)

        # 손실과 R2 score 계산
        loss = weighted_mse_loss(targets, outputs, weights)
        r2 = weighted_r2_score(targets, outputs, weights)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_r2 += r2

        # 배치별 진행상황 출력 (10배치마다)
        if (batch_idx + 1) % 1000 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            avg_r2 = total_r2 / (batch_idx + 1)
            print(f'Batch [{batch_idx+1}/{num_batches}] Loss: {avg_loss:.4f}, R2: {avg_r2:.4f}')

    return total_loss / num_batches, total_r2 / num_batches

def validate(model, valid_loader, device):
    model.eval()
    total_loss = 0
    total_r2 = 0
    num_batches = len(valid_loader)

    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_loader):
            numerical_features = batch['numerical_features'].to(device)
            feature_09 = batch['feature_09'].to(device)
            feature_10 = batch['feature_10'].to(device)
            feature_11 = batch['feature_11'].to(device)
            targets = batch['targets'].to(device)
            weights = batch['weights'].to(device)

            outputs = model(numerical_features)

            loss = weighted_mse_loss(targets, outputs, weights)
            r2 = weighted_r2_score(targets, outputs, weights)

            total_loss += loss.item()
            total_r2 += r2

        if (batch_idx + 1) % 1000 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            avg_r2 = total_r2 / (batch_idx + 1)
            print(f'Batch [{batch_idx+1}/{num_batches}] Loss: {avg_loss:.4f}, R2: {avg_r2:.4f}')
        
    return total_loss / num_batches, total_r2 / num_batches

In [6]:
import polars as pl
train = pl.scan_parquet("/kaggle/input/js24-preprocessing-create-lags/training.parquet").collect().to_pandas()
valid = pl.scan_parquet("/kaggle/input/js24-preprocessing-create-lags/validation.parquet").collect().to_pandas()

In [7]:
data = train
data.dropna(inplace=True)

In [8]:
data_length = data.shape[0]

train_ratio = 0.9
split_point = int(data_length * train_ratio)

train_df = data[:split_point]
valid_df = data[split_point:]

print(f"train_df Length: {train_df.shape[0]}")
print(f"valid_df Length: {valid_df.shape[0]}")

train_df Length: 31052392
valid_df Length: 3450266


In [9]:
category_mappings = {'feature_09': {2: 0, 4: 1, 9: 2, 11: 3, 12: 4, 14: 5, 15: 6, 25: 7, 26: 8, 30: 9, 34: 10, 42: 11, 44: 12, 46: 13, 49: 14, 50: 15, 57: 16, 64: 17, 68: 18, 70: 19, 81: 20, 82: 21},
                     'feature_10': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 10: 7, 12: 8},
                     'feature_11': {9: 0, 11: 1, 13: 2, 16: 3, 24: 4, 25: 5, 34: 6, 40: 7, 48: 8, 50: 9, 59: 10, 62: 11, 63: 12, 66: 13,
                                    76: 14, 150: 15, 158: 16, 159: 17, 171: 18, 195: 19, 214: 20, 230: 21, 261: 22, 297: 23, 336: 24, 376: 25, 388: 26, 410: 27, 522: 28, 534: 29, 539: 30},
                     'symbol_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19,
                                   20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38},
                     'time_id' : {i : i for i in range(968)}}

In [10]:
def apply_category_mappings(df, category_mappings):
    """
    주어진 매핑 딕셔너리를 사용하여 범주형 변수들을 변환하는 함수

    Args:
        df: 변환할 데이터프레임
        category_mappings: 각 컬럼별 매핑 딕셔너리
    """
    df = df.copy()  # 원본 데이터 보존

    for column, mapping in category_mappings.items():
        # 매핑되지 않은 값이 있는지 체크
        unmapped_values = set(df[column].unique()) - set(mapping.keys())
        if unmapped_values:
            print(f"Warning: {column}에서 매핑되지 않은 값 발견: {unmapped_values}")

        # 매핑 적용
        df[column] = df[column].map(mapping)

        # NA 값 체크
        na_count = df[column].isna().sum()
        if na_count > 0:
            print(f"Warning: {column}에서 {na_count}개의 NA 값 발견")

    return df

# 데이터로더 생성 전에 매핑 적용
train_df = apply_category_mappings(train_df, category_mappings)
valid_df = apply_category_mappings(valid_df, category_mappings)

# 매핑 후 값 범위 확인
for column in category_mappings.keys():
    print(f"\n{column} range after mapping:")
    print(f"Train: {train_df[column].min()} to {train_df[column].max()}")
    print(f"Valid: {valid_df[column].min()} to {valid_df[column].max()}")


feature_09 range after mapping:
Train: 0 to 21
Valid: 0 to 20

feature_10 range after mapping:
Train: 0 to 8
Valid: 0 to 8

feature_11 range after mapping:
Train: 0 to 30
Valid: 0 to 29

symbol_id range after mapping:
Train: 0 to 38
Valid: 0 to 38

time_id range after mapping:
Train: 68 to 967
Valid: 68 to 967


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/root/micromamba/envs/kaggle/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at /opt/conda/conda-bld/pytorch_1729647429097/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [12]:
# 컬럼 정의
numerical_columns = data.columns[data.columns.str.contains('feature')].tolist() + ['time_id', 'symbol_id']
target_columns = ['responder_6', 'responder_7', 'responder_8'] # 예측할 타겟들
weight_columns = ['weight']  # 각 타겟에 대한 가중치 (옵션)

# 데이터로더 생성
train_loader, valid_loader = create_data_loaders(
    train_data=train_df,
    valid_data=valid_df,
    numerical_columns=numerical_columns,
    target_columns=target_columns,
    weight_columns=weight_columns,
    batch_size=2048
)

In [13]:
len(numerical_columns)

81

In [16]:
# 모델 초기화
model = CNN1DModel(
    num_numerical_features=len(numerical_columns),
    n_target=len(target_columns),
).to(device)

In [17]:
 # 학습 루프 수정
num_epochs = 100
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay = 5e-4)

for epoch in range(num_epochs):
    train_loss, train_r2 = train_epoch(model, train_loader, optimizer, device)
    valid_loss, valid_r2 = validate(model, valid_loader, device)

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train R2: {train_r2:.4f}')
    print(f'Valid Loss: {valid_loss:.4f}, Valid R2: {valid_r2:.4f}')
    print('-' * 50)

/root/micromamba/envs/kaggle/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  self.pid = os.fork()


Batch [1000/15163] Loss: 1.2563, R2: -0.0032
